Testing base model

In [ ]:
!pip install torch transformers soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoTokenizer,BarkModel
import soundfile as sf
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load pre-trained Bark model and tokenizer
model_name = "suno/bark"
model = BarkModel.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to generate speech with different emotions
def generate_speech_with_emotion(text, emotion):
    # Add emotion prompt to the text
    emotion_prompt = f"[{emotion}] {text}"
    input_ids = tokenizer(emotion_prompt, return_tensors="pt", padding="max_length", max_length=256).input_ids.to(device)

    # Generate speech
    with torch.no_grad():
        generated_audio = model.generate(input_ids)

    return generated_audio.numpy()

# Example text
text = "Hello, this is a test of the Bark model with different emotions."

# List of emotions to test
emotions = ["happy", "sad", "angry", "surprised", "neutral"]

# Generate and save speech for each emotion
for emotion in emotions:
    print(f"Generating speech with emotion: {emotion}")
    audio = generate_speech_with_emotion(text, emotion)
    output_file = f"generated_audio_{emotion}.wav"
    sf.write(output_file, audio, samplerate=22050)
    print(f"Saved {output_file}")

print("Speech generation with different emotions completed.")



Using device: cuda


/usr/local/lib/python3.11/dist-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


Generating speech with emotion: happy


LibsndfileError: Error opening 'generated_audio_happy.wav': Format not recognised.

In [ ]:
# Import necessary libraries
from transformers import AutoProcessor, AutoModel
import torch

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the processor and model
processor = AutoProcessor.from_pretrained("suno/bark")
if processor.tokenizer.pad_token is None:
    processor.tokenizer.pad_token = processor.tokenizer.eos_token

# Move the model to GPU
model = AutoModel.from_pretrained("suno/bark").to(device)


Using device: cuda


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


tensor([[ 0.0005, -0.0001,  0.0002,  ..., -0.0023, -0.0106, -0.0042]])


In [ ]:

# Prepare inputs and move them to GPU
inputs = processor(
    text=["Hello, my name is Nidhi. And, uh — and I like pizza. [laughs] But I also have other interests such as playing tic tac toe."],
    return_tensors="pt",
).to(device)

speech_values = model.generate(**inputs, do_sample=True)

# # Generate speech using the GPU
# with torch.no_grad():
#     speech_values = model.generate(
#         **inputs,
#         semantic_max_new_tokens=100,  # Increase the number of tokens
#         do_sample=True,               # Enable sampling for better diversity
#         top_k=50,                     # Limit sampling to top-k tokens
#         top_p=0.95,                   # Nucleus sampling (top-p)
#         temperature=0.7)         # Control randomness


# # Move the output back to CPU if needed (e.g., for saving or further processing)
# speech_values = speech_values.cpu()

# # Save or process the output
# print(speech_values)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [ ]:
from IPython.display import Audio

sampling_rate = model.generation_config.sample_rate
Audio(speech_values.cpu().numpy().squeeze(), rate=sampling_rate)


In [ ]:
from bark import SAMPLE_RATE, generate_audio, preload_models
audio_array = generate_audio(text_prompt)
Audio(audio_array, rate=SAMPLE_RATE)

NameError: name 'generate_audio' is not defined